# RGB to GrayScale AutoEncoder
In this project, we are going to test the power of AutoEncoder for converting the RGB images to grayscale.

In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class FlowerAutoEncoder(nn.Module):
    def __init__(self):
        super(FlowerAutoEncoder, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, padding = 1)
        self.conv2 = nn.Conv2d(16, 32, 3, padding = 1)
        self.conv3 = nn.Conv2d(32, 64, 3, padding = 1)
        self.conv4 = nn.Conv2d(64, 32, 3, padding = 1)
        ## 128 -> 64 -> 32 -> 16 
        self.pool = nn.MaxPool2d(2, 2)
        
        self.t_conv1 = nn.ConvTranspose2d(32, 64, 2, stride=2)
        self.t_conv2 = nn.ConvTranspose2d(64, 16, 2, stride=2)
        self.t_conv3 = nn.ConvTranspose2d(16, 1, 2, stride=2)
        ## 16 -> 32 -> 64 -> 128
        
    def forward(self, x):
#        x.view(x.size(0))
        x = F.relu(self.conv1(x))
        x = self.pool(x) #128 -> 64
        x = F.relu(self.conv2(x))
        x = self.pool(x) #64 -> 32
        x = F.relu(self.conv3(x)) ## 32 -> 32
        
        x = F.relu(self.conv4(x))
        x = self.pool(x)   #32 -> 16
        
        
        x = F.relu(self.t_conv1(x))
        x = F.relu(self.t_conv2(x))
        x = F.sigmoid(self.t_conv3(x))
        
        
        return x
    
model = FlowerAutoEncoder()
print(model)

FlowerAutoEncoder(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(64, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (t_conv1): ConvTranspose2d(32, 64, kernel_size=(2, 2), stride=(2, 2))
  (t_conv2): ConvTranspose2d(64, 16, kernel_size=(2, 2), stride=(2, 2))
  (t_conv3): ConvTranspose2d(16, 1, kernel_size=(2, 2), stride=(2, 2))
)


In [10]:
# specify loss function
criterion = nn.MSELoss()

# specify loss function
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [11]:
import torch
import numpy as np
from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader


In [12]:
gray_transform = transforms.Compose([ 
                               transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor()
                            
                               ])

transform = transforms.Compose([transforms.ToTensor(), 
                               ])


color_imgs = datasets.ImageFolder('new_color_images/', transform= transform)
gray_imgs  = datasets.ImageFolder('new_gray_images/', transform= gray_transform)
color_train_loader = DataLoader(dataset=color_imgs, batch_size=32, shuffle=False, num_workers=0)
gray_train_loader  = DataLoader(dataset=gray_imgs,  batch_size=32, shuffle=False, num_workers=0)

In [13]:
import matplotlib.pyplot as plt
%matplotlib inline

In [14]:
train_loss =0.0
for i, data in enumerate(zip(color_train_loader, gray_train_loader)):
    color_imgs, _ = data[0]
    gray_imgs, _  = data[1]
    
    optimizer.zero_grad()                #Z
    outputs = model(color_imgs)           #M
    
    loss = criterion(outputs, gray_imgs) #C
    loss.backward()                      #B
    optimizer.step()                     #S
    train_loss += loss.item()*len(color_imgs)
    if i >100 :
        print("i:", i, "train_loss :", train_loss)
print("Final Loss: ",train_loss/len(color_train_loader))


/opt/anaconda3/lib/python3.7/site-packages/torch/nn/functional.py:1350: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


i: 101 train_loss : 164.4949079155922
i: 102 train_loss : 164.870320469141
i: 103 train_loss : 165.39048072695732
i: 104 train_loss : 165.77197855710983
i: 105 train_loss : 166.2068169414997
i: 106 train_loss : 166.69465002417564
i: 107 train_loss : 167.20185247063637
i: 108 train_loss : 167.6479260623455
i: 109 train_loss : 168.03916016221046
i: 110 train_loss : 168.54111739993095
i: 111 train_loss : 168.9396480023861
i: 112 train_loss : 169.4237543642521
i: 113 train_loss : 169.9024589061737
Final Loss:  1.4903724465453834


In [16]:
total_epochs = 20
for epoch in range(total_epochs +1):
    train_loss =0.0
    for i, data in enumerate(zip(color_train_loader, gray_train_loader)):
        color_imgs, _ = data[0]
        gray_imgs, _  = data[1]

        optimizer.zero_grad()                #Z
        outputs = model(color_imgs)           #M
        loss = criterion(outputs, gray_imgs) #C
        loss.backward()                      #B
        optimizer.step()                     #S
        train_loss += loss.item()*len(color_imgs)
        
    print("Final Loss: ",train_loss/len(color_train_loader))


/opt/anaconda3/lib/python3.7/site-packages/torch/nn/functional.py:1350: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Final Loss:  0.4615851951795712
Final Loss:  0.37972524788296014
Final Loss:  0.349574395569793
Final Loss:  0.32863062973085205
Final Loss:  0.3141281589081413
Final Loss:  0.3033866963365622
Final Loss:  0.29555684458791165
Final Loss:  0.2877801646266067
Final Loss:  0.2824120001311888
Final Loss:  0.27786807689750403
Final Loss:  0.2716353173580086
Final Loss:  0.27110359595533
Final Loss:  0.2642918461770342
Final Loss:  0.2617973895710811
Final Loss:  0.2580045415905484
Final Loss:  0.253218232801086
Final Loss:  0.24965701056154152
Final Loss:  0.2487724896585732
Final Loss:  0.24483934698397652
Final Loss:  0.24210377240128683
Final Loss:  0.2397237484130943
